In [1]:
%run ../../src/feature_selection_scripts/Feature_Selection_Bin_Cla_SRV.ipynb

['Seq', 'sTos', 'sTtl', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'AckDat', 'icmp', 'tcp', 'udp', 'ECO', 'FIN', 'INT', 'RST', 'Start']
(911917, 15)


In [2]:
import pickle
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [3]:
class TabNetClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, tabnet_model):
        self.tabnet_model = tabnet_model

    def fit(self, X, y):
        self.tabnet_model.fit(X, y)
        return self

    def predict(self, X):
        return self.tabnet_model.predict(X)

    def predict_proba(self, X):
        return self.tabnet_model.predict_proba(X)

In [4]:
with open('../AdaBoost/adaboost_clf.pkl', 'rb') as f:
    adaboost_clf = pickle.load(f)

with open('../Light_GBM/lgb_clf.pkl', 'rb') as f:
    lgb_clf = pickle.load(f)

with open('../TabNet_Classifier/tabnet_clf.pkl', 'rb') as f:
    tabnet_clf = pickle.load(f)

with open('../SVM/svm_clf.pkl', 'rb') as f:
    svm_clf = pickle.load(f)

with open('../CatBoost/catboost_clf.pkl', 'rb') as f:
    catboost_clf = pickle.load(f)

with open('../XGBoost/xgb_clf.pkl', 'rb') as f:
    xgb_clf = pickle.load(f)

In [5]:
X = df[selected_feature_labels].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [6]:
tabnet_wrapper = TabNetClassifierWrapper(tabnet_clf)

In [7]:
classifiers = [('AdaBoost', adaboost_clf), 
               ('XGBoost', xgb_clf),
               ('CatBoost', catboost_clf),
               ('SVM', svm_clf),
               ('LightGBM', lgb_clf)]

voting_clf = VotingClassifier(estimators=classifiers, voting='hard')

voting_clf.fit(X_train, y_train)

y_pred = voting_clf.predict(X_test)

/opt/homebrew/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Learning rate set to 0.189162
0:	learn: 0.2111311	total: 109ms	remaining: 1m 49s
1:	learn: 0.0664737	total: 151ms	remaining: 1m 15s
2:	learn: 0.0267207	total: 191ms	remaining: 1m 3s
3:	learn: 0.0143874	total: 228ms	remaining: 56.7s
4:	learn: 0.0088684	total: 266ms	remaining: 52.8s
5:	learn: 0.0068808	total: 305ms	remaining: 50.5s
6:	learn: 0.0061003	total: 343ms	remaining: 48.6s
7:	learn: 0.0051481	total: 383ms	remaining: 47.5s
8:	learn: 0.0042929	total: 422ms	remaining: 46.5s
9:	learn: 0.0039921	total: 462ms	remaining: 45.8s
10:	learn: 0.0037242	total: 497ms	remaining: 44.7s
11:	learn: 0.0036427	total: 538ms	remaining: 44.3s
12:	learn: 0.0034173	total: 591ms	remaining: 44.8s
13:	learn: 0.0032929	total: 628ms	remaining: 44.2s
14:	learn: 0.0030571	total: 665ms	remaining: 43.6s
15:	learn: 0.0028141	total: 705ms	remaining: 43.4s
16:	learn: 0.0025549	total: 747ms	remaining: 43.2s
17:	learn: 0.0024895	total: 780ms	remaining: 42.5s
18:	learn: 0.0023903	total: 819ms	remaining: 42.3s
19:	learn

In [8]:
accuracy = accuracy_score(y_test, y_pred)
precision_benign = precision_score(y_test, y_pred, pos_label=0)
recall_benign = recall_score(y_test, y_pred, pos_label=0)
f1_benign = f1_score(y_test, y_pred, pos_label=0)
precision_malicious = precision_score(y_test, y_pred, pos_label=1)
recall_malicious = recall_score(y_test, y_pred, pos_label=1)
f1_malicious = f1_score(y_test, y_pred, pos_label=1)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("precision_benign:", precision_benign)
print("recall_benign:", recall_benign)
print("f1_benign:", f1_benign)
print("precision_malicious:", precision_benign)
print("recall_malicious:", recall_benign)
print("f1_malicious:", f1_malicious)

Accuracy: 0.9996414155204574
precision_benign: 0.9993062636866652
recall_benign: 0.9997825814274366
f1_benign: 0.9995443658115514
precision_malicious: 0.9993062636866652
recall_malicious: 0.9997825814274366
f1_malicious: 0.9997043819277011


In [9]:
with open('voting_clf.pkl', 'wb') as f:
    pickle.dump(voting_clf, f)